### Testes utilizando transforms diretamente

#### Vision

In [9]:
import os
import base64
import logging
from time import sleep
from openai import OpenAI
from dotenv import load_dotenv

In [10]:
# Configuração do ambiente e logging
load_dotenv()
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

In [11]:
# Configurações do OpenAI
cliente = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
modelo = "gpt-4o-mini"

In [12]:
# Diretórios
BASE_DIR = os.getcwd()
DIR_PAI = os.path.dirname(BASE_DIR)
DIR_DATA = os.path.join(DIR_PAI, "data")
DIR_PDF_TO_IMAGE = os.path.join(DIR_DATA, "processed_pdf_to_images")
DIR_VISION = os.path.join(DIR_DATA, "outputs_vision")

In [13]:
# Função para codificar imagens em Base64
def encodar_imagem(caminho_imagem):
    """Codifica uma imagem em Base64."""
    try:
        with open(caminho_imagem, "rb") as arquivo_imagem:
            return base64.b64encode(arquivo_imagem.read()).decode("utf-8")
    except Exception as e:
        logging.error(f"Erro ao codificar imagem {caminho_imagem}: {e}")
        return None

In [14]:
# Função para análise de imagem
def analisar_imagem(caminho_imagem):
    """Analisa a imagem usando o Vision da OpenAI."""
    logging.info(f"Iniciando análise da imagem: {caminho_imagem}")
    imagem_base64 = encodar_imagem(caminho_imagem)
    if not imagem_base64:
        return None, 0

    # Prompt para análise
    prompt = f"""
    Analise o conteúdo técnico da imagem fornecida. Extraia e organize informações relevantes, incluindo:
    - Títulos, subtítulos e parágrafos.
    - Elementos visuais como gráficos, tabelas e diagramas.
    - Procedimentos e especificações técnicas.
    Seja claro, preciso e mantenha a hierarquia das informações.
    ,{imagem_base64}
    """

    try:
        resposta = cliente.chat.completions.create(
            model=modelo,
            messages=[
                {
                "role": "user",
                "content": [
                    {
                        "type": "text", "text": prompt
                    },
                    {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{imagem_base64}",
                    },
                    },
                ],
                }
            ],
            max_tokens=600,
            )
        conteudo = resposta.choices[0].message.content
        logging.info("Análise concluída com sucesso.")
        return conteudo, 0  # Retorna a descrição e custo fictício por enquanto
    except Exception as e:
        logging.error(f"Erro ao processar a imagem {caminho_imagem}: {e}")
        return None, 0

In [15]:
# Função para salvar resultado
def salvar_resultado(nome_do_arquivo, conteudo):
    """Salva o conteúdo em um arquivo texto."""
    try:
        with open(nome_do_arquivo, "w", encoding="utf-8") as arquivo:
            arquivo.write(conteudo)
        logging.info(f"Resultado salvo em {nome_do_arquivo}")
    except Exception as e:
        logging.error(f"Erro ao salvar o arquivo {nome_do_arquivo}: {e}")

In [16]:
# Execução principal
if __name__ == "__main__":
    total_custo = 0

    # Iterar pelas subpastas de DIR_PDF_TO_IMAGE
    for subpasta in os.listdir(DIR_PDF_TO_IMAGE):
        subpasta_path = os.path.join(DIR_PDF_TO_IMAGE, subpasta)
        if os.path.isdir(subpasta_path):  # Verifica se é uma pasta
            logging.info(f"Processando subpasta: {subpasta}")

            # Criar pasta correspondente em DIR_VISION
            vision_output_path = os.path.join(DIR_VISION, subpasta)
            os.makedirs(vision_output_path, exist_ok=True)

            # PROCESSAR CADA IMAGEM NAS SUBPASTAS CERTAS
            for arquivo in os.listdir(subpasta_path):
                if arquivo.lower().endswith((".jpg", ".jpeg", ".png")):
                    caminho_imagem = os.path.join(subpasta_path, arquivo)
                    try:
                        descricao, _ = analisar_imagem(caminho_imagem)
                        if descricao:
                            # SALVANDO RESULTADO
                            nome_do_arquivo = os.path.join(vision_output_path, f"{os.path.splitext(arquivo)[0]}_description.txt")
                            salvar_resultado(nome_do_arquivo, descricao)
                    except Exception as e:
                        logging.error(f"Erro no processament o da imagem {arquivo}: {e}")
                    sleep(1)  # LIMITE DE REQUISIÇÃO
    logging.info("Processamento concluído.")

2024-12-09 09:15:53,950 - INFO - Processando subpasta: 1 - Engine Control-Fuel System
2024-12-09 09:15:53,951 - INFO - Iniciando análise da imagem: c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\processed_pdf_to_images\1 - Engine Control-Fuel System\1 - Engine Control-Fuel System_pag1.jpg
2024-12-09 09:16:06,218 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-09 09:16:06,220 - INFO - Análise concluída com sucesso.
2024-12-09 09:16:06,221 - INFO - Resultado salvo em c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\outputs_vision\1 - Engine Control-Fuel System\1 - Engine Control-Fuel System_pag1_description.txt
2024-12-09 09:16:07,222 - INFO - Iniciando análise da imagem: c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\processed_pdf_to_images\1 - Engine Control-Fuel System\1 - Engine Control-Fuel System_pag10.jpg
2024-12-09 09:16:19,556 - INFO - HTTP Request: POST https://api.open

KeyboardInterrupt: 